<center>
  <font size="7">Tests unitaires de l'API</font><br>
  <font size="5">Projet 7 - Implémentez un modèle de scoring</font>
</center>
<div align="right">
  <font size="4"><i>par Jean Vallée</i></font>
</div>

<hr size=5>

# Scénario de test unitaire 
Vu qu'il s'agit d'un modèle de classification binaire, le test unitaire consistera à :
- faire prédire la variable cible par l'API du modèle 
- fournir au modèle les valeurs des attributs via un formulaire

Le micro-framework Web léger [_Flask_](https://flask.palletsprojects.com/en/3.0.x/quickstart/) prend en charge la gestion de requêtes et réponses REST. Cf. section [Serveur Web](#web_server)

In [61]:
ip_server = '4.233.201.217/'
port_staging_server, port_web_server = '5677', '6543'
url_staging_server = ip_server + ':' + port_staging_server + '/invocations'

# Tests avec _pytest_

## Installation de _pytest_

In [5]:
! pip install pytest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 KB 6.4 MB/s eta 0:00:00a 0:00:01


## Fonctions de test

### Test de connexion au serveur

In [62]:
dir_test_data = '../test_api/data/'
path_test_json = dir_test_data + 'dict_X_single.json'
path_test_1 = './test_1.py'

In [63]:
str_test_1 = \
'''\
import requests
def test_connection():    
    print('__________Connection to API Server_____________')
    req_post = requests.post(   url     = 'http://ip_server:port_server/invocations', 
                                headers = {'Content-Type': 'application/json'}, 
'''
str_test_1 = str_test_1 .replace('ip_server',   ip_server) \
                        .replace('port_server', port_staging_server)

In [64]:
with open(path_test_json) as file_object:
    str_test_values = file_object.read()
str_test_values = str_test_values.replace('[', '').replace(']\n', '')
str_test_values

'{"CODE_GENDER_M":1,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATION_TYPE_Lower_secondary":0,"EXT_SOURCE_3":1.0,"NAME_EDUCATION_TYPE_Higher_education":1,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0}'

In [65]:
str_test_1 += \
'''\
                                data    = '{"inputs": str_test_values}' )  \
'''   
str_test_1 = str_test_1.replace('str_test_values', str_test_values)

In [67]:
str_test_1 += \
'''
    is_connected = req_post.ok
    if is_connected : print('OK : Connected to API Server')
    assert is_connected
'''

In [68]:
with open(path_test_1, "w") as file_object:
    print(str_test_1, file=file_object)

In [69]:
!cat $path_test_1

import requests
def test_connection():    
    print('__________Connection to API Server_____________')
    req_post = requests.post(   url     = 'http://4.233.201.217/:5677/invocations', 
                                headers = {'Content-Type': 'application/json'}, 
                                data    = '{"inputs": {"CODE_GENDER_M":1,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATION_TYPE_Lower_secondary":0,"EXT_SOURCE_3":1.0,"NAME_EDUCATION_TYPE_Higher_education":1,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0}}' )  
    is_connected = req_post.ok
    if is_connected : print('OK : Connected to API Server')
    assert is_connected



### Tests unitaires

In [71]:
file_dict_X_sample = dir_test_data + 'dict_X_sample.json'
file_list_y_sample = dir_test_data + 'li_y_sample.txt'
path_test_2 = './test_2.py'

In [72]:
str_test_2 = \
'''\
import requests
import json

def predict(dict_features):
    # Send input data to prediction API
    req_post = requests.post(   url     = 'http://ip_server:port_server/invocations',
                                headers = {'Content-Type': 'application/json'},
                                data    = json.dumps({'inputs': dict_features}) )
    # Get predicted value from API
    dict_predicted = json.loads(req_post.text)
    return dict_predicted

def test_prediction():
'''
str_test_2 = str_test_2 .replace('ip_server',   ip_server) \
                        .replace('port_server', port_staging_server)

In [73]:
with open(file_dict_X_sample) as file_object:
    str_li_dict_features = file_object.read()
str_li_dict_features = str_li_dict_features.replace('\n', '')
li_dict_features = eval(str_li_dict_features)
li_dict_features

[{'CODE_GENDER_M': 0,
  'NAME_CONTRACT_TYPE_Cash_loans': 1,
  'NAME_EDUCATION_TYPE_Lower_secondary': 1,
  'EXT_SOURCE_3': 0.1359510442,
  'NAME_EDUCATION_TYPE_Higher_education': 0,
  'NAME_EDUCATION_TYPE_Secondary_or_secondary_special': 0},
 {'CODE_GENDER_M': 0,
  'NAME_CONTRACT_TYPE_Cash_loans': 1,
  'NAME_EDUCATION_TYPE_Lower_secondary': 1,
  'EXT_SOURCE_3': 0.244516392,
  'NAME_EDUCATION_TYPE_Higher_education': 0,
  'NAME_EDUCATION_TYPE_Secondary_or_secondary_special': 0},
 {'CODE_GENDER_M': 0,
  'NAME_CONTRACT_TYPE_Cash_loans': 1,
  'NAME_EDUCATION_TYPE_Lower_secondary': 1,
  'EXT_SOURCE_3': 0.2485355573,
  'NAME_EDUCATION_TYPE_Higher_education': 0,
  'NAME_EDUCATION_TYPE_Secondary_or_secondary_special': 0}]

In [74]:
nb_observations = len(li_dict_features)
li_targets = [1] * nb_observations
str_li_targets = str(li_targets)
str_li_targets

'[1, 1, 1]'

In [75]:
str_test_2 += \
'''\
    li_dict_features = str_li_dict_features
    li_targets = str_li_targets
    for idx, (dict_features_i, out_i) in enumerate(zip(li_dict_features, li_targets)) :
'''   
str_test_2 = str_test_2 .replace('str_li_dict_features', str_li_dict_features) \
                        .replace('str_li_targets', str_li_targets)

In [76]:
str_test_2 += \
'''
        dict_predicted_i = predict(dict_features_i)
        dict_expected_i  = {'predictions':[out_i]}
        print('input features   :', dict_features_i)
        print('output predicted:', dict_predicted_i)
        print('output expected :', dict_expected_i)

        # Compare predicted & expected values
        assert dict_predicted_i == dict_expected_i
'''

In [77]:
with open(path_test_2, "w") as file_object:
    print(str_test_2, file=file_object)

In [78]:
!cat $path_test_2

import requests
import json

def predict(dict_features):
    # Send input data to prediction API
    req_post = requests.post(   url     = 'http://4.233.201.217/:5677/invocations',
                                headers = {'Content-Type': 'application/json'},
                                data    = json.dumps({'inputs': dict_features}) )
    # Get predicted value from API
    dict_predicted = json.loads(req_post.text)
    return dict_predicted

def test_prediction():
    li_dict_features = [{"CODE_GENDER_M":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATION_TYPE_Lower_secondary":1,"EXT_SOURCE_3":0.1359510442,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0},{"CODE_GENDER_M":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATION_TYPE_Lower_secondary":1,"EXT_SOURCE_3":0.244516392,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0},{"CODE_GENDER_M":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATI

## Exécution et résultats des tests

In [23]:
!python -m pytest --import-mode=append -rA ../test_api/

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.2.2, pluggy-1.5.0
rootdir: /home/jvisa4031/project_7/test_api
plugins: anyio-4.4.0, Faker-25.5.0
collected 2 items                                                              

test_1.py F                                                              [ 50%]
test_2.py F                                                              [100%]

=================================== FAILURES ===================================
_______________________________ test_connection ________________________________

self = <urllib3.connection.HTTPConnection object at 0x7a8d67fad450>

    def _new_conn(self) -> socket.socket:
        """Establish a socket connection and set nodelay settings on it.
    
        :return: New socket connection.
        """
        try:
>           sock = connection.create_connection(
                (self._dns_host, self.port),
                self.timeo

**Observation** : les résultats des tests unitaires sont positifs

# Tests via _curl_

In [24]:
import subprocess

Récupération de la liste d'attributs

In [25]:
with open(dir_test_data + 'li_features.txt') as file_object:
    str_li_features = file_object.read()
str_li_features = str_li_features.replace('\'', '"').replace('\n', '')
str_li_features

'["CODE_GENDER_M", "NAME_CONTRACT_TYPE_Cash_loans", "NAME_EDUCATION_TYPE_Lower_secondary", "EXT_SOURCE_3", "NAME_EDUCATION_TYPE_Higher_education", "NAME_EDUCATION_TYPE_Secondary_or_secondary_special"]'

In [26]:
def get_single_prediction(str_values, str_expected) :
    shell_command = 'curl -d \'{"dataframe_split": { "columns": ' + str_li_features \
        + ', "data": [[' + str_values + ']]}}\' -H \'Content-Type: application/json\' -X POST ' \
        + url_staging_server + ' --no-progress-meter'
    print('Shell command :', shell_command, '\n')
    print(subprocess.getoutput(shell_command), 'expected :', str_expected, 'for features values:', str_values)

In [27]:
get_single_prediction('1, 1, 0, 1, 1, 0', '0')

Shell command : curl -d '{"dataframe_split": { "columns": ["CODE_GENDER_M", "NAME_CONTRACT_TYPE_Cash_loans", "NAME_EDUCATION_TYPE_Lower_secondary", "EXT_SOURCE_3", "NAME_EDUCATION_TYPE_Higher_education", "NAME_EDUCATION_TYPE_Secondary_or_secondary_special"], "data": [[1, 1, 0, 1, 1, 0]]}}' -H 'Content-Type: application/json' -X POST 4.233.201.217/:5677/invocations --no-progress-meter 

curl: (28) Failed to connect to 4.233.201.217 port 80 after 134738 ms: Connection timed out expected : 0 for features values: 1, 1, 0, 1, 1, 0


# Tests via interface Web d'API

### Accès en ligne

In [28]:
print('Website URL =' , 'http://' + ip_server + ':' + port_web_server)

Website URL = http://4.233.201.217/:6543


### Arborescence de fichiers

In [29]:
! find ../website | sed -e "s/[^\/]*\//  |/g" -e "s/|\([^ ]\)/└── \1/"

  └── website
  |  └── static
  |  |  └── css
  |  |  |  └── style.css
  |  └── web_router.py
  |  └── templates
  |  |  └── form.html
  |  |  └── report.html
  |  |  └── index.html
  |  |  └── result.html
  |  └── instructions_WebServer.md


### Fichiers _HTML_

In [30]:
import IPython
dir_website = '../website/'
dir_html = dir_website + 'templates/'

#### Index  
Fichier d'accueil

In [31]:
file_html_index = dir_html + 'index.html'
!cat $file_html_index


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page d'accueil</h2>    
       <a href="/form/">Formulaire</a>
    </body>
</html>



In [32]:
IPython.display.IFrame(file_html_index, width=700, height=350) 

#### Formulaire  
Formulaire de saisie des atttributs

In [33]:
path_form = dir_html + 'form.html'

In [34]:
str_form = \
'''\
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
        <h1>API du modèle de classification binaire</h1>       
        <h2>Formulaire de saisie d'attributs</h2>       
        
        <form method="post" action="{{ url_for('result') }}">
            <div>
            	<label for="port">Server:</label>
            	<select id="port" name="port">
                		<option value="5677"> Staging    </option>
                		<option value="5678"> Production </option>
            	</select>
            </div>
            <br>
            <table> \
'''

In [35]:
li_features = eval(str_li_features)
li_features

['CODE_GENDER_M',
 'NAME_CONTRACT_TYPE_Cash_loans',
 'NAME_EDUCATION_TYPE_Lower_secondary',
 'EXT_SOURCE_3',
 'NAME_EDUCATION_TYPE_Higher_education',
 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special']

In [36]:
with open(dir_test_data + 'li_types.txt') as file_object:
    str_li_types = file_object.read()
str_li_types = str_li_types.replace('[', '["').replace(', ', '", "').replace(']\n', '"]')
li_types = eval(str_li_types)
li_types

['long', 'long', 'long', 'double', 'long', 'long']

In [37]:
def get_div_per_feature(idx, str_feature_i, str_type_i) :
    str_div_i = \
    '''
                <tr>
                    <td><label for="input_str_idx">str_feature_i :</label></td>
                    <td><input name="str_feature_i" id="input_str_idx" type="number" step="any" value="1"></td>
                    <td>str_type_i</td>
                </tr> \
    '''
    return str_div_i.replace('str_idx',       str(idx))      \
                    .replace('str_feature_i', str_feature_i) \
                    .replace('str_type_i',    str_type_i)

In [38]:
for idx, (feature_i, type_i) in enumerate(zip(li_features, li_types)) :
    str_form += get_div_per_feature(idx + 1, feature_i, type_i)

In [39]:
str_form += \
'''
            </table>
            <br>
            <div>
                <button>Predict</button>
            </div>
        </form>
        <br>
        <p>PI, valeurs des attributs pour obtenir des TP (vrai positif):</p>
        str_pandas_as_html
    </body>
</html>
'''

In [40]:
import pandas as pd
file_X_true_positives = '../modeling/data/out/X_true_pos.csv'
df_X_true_pos = pd.read_csv(file_X_true_positives).drop('Unnamed: 0', axis='columns')

In [41]:
str_pandas_as_html = df_X_true_pos.to_html(justify='center', border=0,
                                float_format=lambda x: '%.10f' % x)
str_form = str_form.replace('str_pandas_as_html', str_pandas_as_html)

In [42]:
with open(path_form, 'w') as file_object:
    print(str_form, file=file_object)

In [43]:
!cat $path_form

<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
        <h1>API du modèle de classification binaire</h1>       
        <h2>Formulaire de saisie d'attributs</h2>       
        
        <form method="post" action="{{ url_for('result') }}">
            <div>
            	<label for="port">Server:</label>
            	<select id="port" name="port">
                		<option value="5677"> Staging    </option>
                		<option value="5678"> Production </option>
            	</select>
            </div>
            <br>
            <table> 
                <tr>
                    <td><label for="input_1">CODE_GENDER_M :</label></td>
                    <td><input name="CODE_GENDER_M" id="input_1" type="number" step="any" value="1"></td>
                    <td>long</td>
                </tr>     
           

In [44]:
IPython.display.IFrame(path_form, width=700, height=350) 

#### Résultat
Page de résultat de prédiction par l'API du modèle

In [45]:
path_result = dir_html + 'result.html'

In [46]:
str_result = \
'''\
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page de prédiction du modèle</h2> 
       <h3>Valeur prédite = {{target_value['predictions']}}</h3>
       <p>Pour les valeurs saisies des attributs suivants :</p>
       <table>
'''

In [47]:
li_features

['CODE_GENDER_M',
 'NAME_CONTRACT_TYPE_Cash_loans',
 'NAME_EDUCATION_TYPE_Lower_secondary',
 'EXT_SOURCE_3',
 'NAME_EDUCATION_TYPE_Higher_education',
 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special']

In [48]:
def get_div_per_feature(idx, str_feature_i) :
    str_div_i = \
    ''' 
            <tr>
                <td>str_feature_i</td><td>{{features['str_feature_i']}}</td>  
            </tr> \
    '''
    return str_div_i.replace('str_idx',       str(idx))    \
                    .replace('str_feature_i', str_feature_i)                        

In [49]:
for idx, feature_i in enumerate(li_features) :
    str_result += get_div_per_feature(idx + 1, feature_i)

In [50]:
str_result += \
'''
       </table>
       <br>
       <a href="/">Retour à l'accueil</a>
       <a href="/form/">Formulaire</a>
    </body>
</html>
'''

In [51]:
with open(path_result, "w") as file_object:
    print(str_result, file=file_object)

In [52]:
!cat $path_result

<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page de prédiction du modèle</h2> 
       <h3>Valeur prédite = {{target_value['predictions']}}</h3>
       <p>Pour les valeurs saisies des attributs suivants :</p>
       <table>
 
            <tr>
                <td>CODE_GENDER_M</td><td>{{features['CODE_GENDER_M']}}</td>  
            </tr>      
            <tr>
                <td>NAME_CONTRACT_TYPE_Cash_loans</td><td>{{features['NAME_CONTRACT_TYPE_Cash_loans']}}</td>  
            </tr>      
            <tr>
                <td>NAME_EDUCATION_TYPE_Lower_secondary</td><td>{{features['NAME_EDUCATION_TYPE_Lower_secondary']}}</td>  
            </tr>      
            <tr>
                <td>EXT_SOURCE_3</td><td>{{features['EXT_SOURCE_3']}}

In [53]:
IPython.display.IFrame(path_result, width=700, height=350) 

#### Style

In [54]:
dir_style = './static/css/'

In [55]:
file_style = dir_style + 'style.css'
!cat $file_style

cat: ./static/css/style.css: No such file or directory


### Fichiers _Python_

#### Routeur Web  
Un fichier en Pyhton couvre la gestion REST

In [56]:
web_router = dir_website + 'web_router.py'
!cat $web_router

from flask import Flask, render_template, request
import requests
import json
import subprocess
 
def run_shell(command):
    shell_process = subprocess.run([command], shell=True, capture_output=True, text=True)
    return str(shell_process.stdout) + str(shell_process.stderr)
def pull():
    dir_root = '/home/azureuser/project_7/'
    str_command_pull = 'cd ' + dir_root + ' ; git pull origin main'
    str_output = run_shell(str_command_pull)
    return 'Git Pull Model:===============\n' + str_output
def restart(str_environment) : 
    if str_environment == 'staging'    : dir_model, port = '../api/staging_model/',    '5677'
    if str_environment == 'production' : dir_model, port = '../api/production_model/', '5678' 
    str_command_serve = 'mlflow models serve -m ' + dir_model + ' -p ' + port + ' -h 0.0.0.0 --no-conda &'
    str_command_ps = 'ps aux | grep  ":' + port + '" | grep -v grep | awk \'{print $2, $15, $19}\' '
    str_output = 'Process BEFORE Restart:\n'                    # 

## Serveur _Web_

<a id="web_server"></a>
<hr>

### Lancement

In [57]:
shell_command = 'flask --app $web_router run -h 0.0.0.0 -p 6543 --debug > ./flask_app.log 2>&1 &'
get_ipython().system_raw(shell_command) # run model API in background

### Vérifications
#### Log

In [58]:
!tail ./flask_app.log

#### Processus
Liste des processus

In [59]:
li_ps = !ps aux | grep "flask" | grep -v "grep" | awk '{print $2}' 
!ps aux | grep "flask" | grep -v "grep" 

jvisa40+    6826  0.0  0.1  37248 28064 ?        R    18:03   0:00 /home/jvisa4031/environments_folder/my_env/bin/python3 /home/jvisa4031/environments_folder/my_env/bin/flask --app ../website/web_router.py run -h 0.0.0.0 -p 6543 --debug


Arrêt des processus

In [60]:
for ps_i in li_ps : 
    #!kill -9 $ps_i

SyntaxError: incomplete input (2464387668.py, line 2)